<a href="https://colab.research.google.com/github/danieltalon/trabalho_RAG/blob/main/classifica_perguntas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- PASSO 1: Instalar a biblioteca ---
!pip install -q -U google-generativeai

# --- PASSO 2: Importações e Configuração da API Key ---
import json
import textwrap
import google.generativeai as genai
from google.colab import userdata
import time
import ast
from collections import Counter
import os
from google.api_core import exceptions as google_api_exceptions
import requests

# --- BUSCAR A API KEY DOS SECRETS DO COLAB ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API Key não encontrada ou vazia.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key carregada com sucesso.")
except Exception as e:
    print(f"Erro ao carregar a API Key: {e}")
    print("Por favor, certifique-se de que adicionou a GOOGLE_API_KEY aos Secrets do Colab (ícone de chave à esquerda) e ativou o 'Notebook access'.")
    raise SystemExit("API Key não configurada.")

# --- Configurações do Modelo Gemini ---
MODEL_NAME = 'gemini-1.5-flash'
model = genai.GenerativeModel(MODEL_NAME)

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

generation_config = genai.GenerationConfig(
    temperature=0.3,
    top_p=0.9,
    top_k=40
)

# --- PASSO 3: Ler o JSON de Entrada ---
input_file_path = '/content/questoes-Revalida+Fuvest.json'
# --- MODIFICAÇÃO: Nome do arquivo de saída específico para 2024 ---
output_file_path = '/content/questoes_2024_classificadas.json'
all_data = [] # Lista para armazenar todos os dados carregados

print(f"\nTentando ler o arquivo de entrada: {input_file_path}")
if not os.path.exists(input_file_path):
    print(f"Erro: Arquivo '{input_file_path}' não encontrado.")
    print("Certifique-se de que o arquivo foi carregado corretamente no Colab.")
else:
    try:
        with open(input_file_path, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
        print(f"Arquivo '{input_file_path}' lido com sucesso. {len(all_data)} questões encontradas no total.")
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{input_file_path}' não é um JSON válido.")
        all_data = []
    except Exception as e:
        print(f"Erro inesperado ao ler o arquivo: {e}")
        all_data = []

# --- MODIFICAÇÃO: Filtrar os dados para o ano de 2024 ---
data_to_process = []
target_year = 2024
if all_data:
    data_to_process = [q for q in all_data if q.get('prova') == target_year]
    print(f"\nFiltrado para o ano {target_year}. {len(data_to_process)} questões encontradas para processar.")
    if not data_to_process:
        print(f"Nenhuma questão encontrada para o ano {target_year}. O script será encerrado sem classificação.")
else:
    print("Nenhum dado carregado do arquivo JSON.")

# --- PASSO 4: Iterar e Classificar (APENAS os dados filtrados) ---
if data_to_process: # Só executa se houver questões de 2024
    print("\nIniciando classificação das questões de 2024...")
    area_counter = Counter()
    total_questoes_filtradas = len(data_to_process)
    questoes_processadas = 0
    erros_api = 0
    erros_parse = 0

    # Pausa entre requisições (ajuste conforme necessário para evitar erro 429)
    PAUSA_ENTRE_REQUISICOES = 5 # segundos (12 RPM) - Aumente se necessário

    MAX_TENTATIVAS_CONEXAO = 3

    # --- MODIFICAÇÃO: Iterar sobre data_to_process ---
    for i, questao in enumerate(data_to_process):
        print(f"Classificando questão {i+1}/{total_questoes_filtradas} (Prova {questao.get('prova', 'N/A')} - Nº {questao.get('numero', 'N/A')})...", end="")
        tentativa_atual = 0
        sucesso_api = False
        response = None

        while tentativa_atual < MAX_TENTATIVAS_CONEXAO and not sucesso_api:
            tentativa_atual += 1
            try:
                enunciado = questao.get('enunciado')
                if not enunciado:
                    print(" [ERRO: Enunciado vazio. Pulando]")
                    questao['areas_medicas'] = ['Não Classificado - Enunciado Vazio']
                    erros_parse += 1
                    sucesso_api = True
                    continue

                prompt = f"""
                Analise o enunciado da questão médica abaixo e classifique-o em uma ou mais áreas principais da medicina.
                Exemplos de áreas: Pediatria, Obstetrícia, Ginecologia, Cardiologia, Cirurgia Geral, Clínica Médica, Preventiva/Saúde Coletiva, Infectologia, Psiquiatria, Dermatologia, Neurologia, Ortopedia, Oftalmologia, Otorrinolaringologia, Nefrologia, Endocrinologia, Gastroenterologia, Pneumologia, Hematologia, Oncologia, Ética Médica, Medicina Legal, Saúde do Trabalhador, Geriatria, Urgência/Emergência.

                Se a questão abordar múltiplos temas, liste todas as áreas relevantes. Se for muito geral ou difícil de classificar especificamente, use 'Clínica Médica Geral'.

                Retorne APENAS uma lista Python de strings contendo as áreas identificadas. Exemplo de formato de saída: ['Pediatria', 'Infectologia']

                Enunciado da Questão:
                "{enunciado}"

                Classificação (apenas a lista Python):
                """

                response = model.generate_content(
                    prompt,
                    generation_config=generation_config,
                    safety_settings=safety_settings
                )
                sucesso_api = True

            except google_api_exceptions.ResourceExhausted as e:
                print(f" [ERRO API - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_CONEXAO}: Quota Excedida (429). Esperando {PAUSA_ENTRE_REQUISICOES * 5}s...]", end="")
                time.sleep(PAUSA_ENTRE_REQUISICOES * 5)
                if tentativa_atual == MAX_TENTATIVAS_CONEXAO:
                    print(" [ERRO API: Máximo de tentativas atingido para Quota Excedida.]")
                    questao['areas_medicas'] = ['Erro API - Quota Excedida']
                    erros_api += 1
            except (requests.exceptions.ConnectionError, google_api_exceptions.ServiceUnavailable, ConnectionResetError) as e:
                print(f" [ERRO API - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_CONEXAO}: Erro de conexão ({type(e).__name__}). Esperando {PAUSA_ENTRE_REQUISICOES}s...]", end="")
                time.sleep(PAUSA_ENTRE_REQUISICOES)
                if tentativa_atual == MAX_TENTATIVAS_CONEXAO:
                    print(" [ERRO API: Máximo de tentativas atingido para Erro de Conexão.]")
                    questao['areas_medicas'] = ['Erro API - Conexão']
                    erros_api += 1
            except Exception as e:
                print(f" [ERRO GERAL INESPERADO - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_CONEXAO}: {type(e).__name__} - {e}]")
                questao['areas_medicas'] = ['Erro na Classificação - Inesperado']
                erros_api += 1
                sucesso_api = True
                time.sleep(PAUSA_ENTRE_REQUISICOES * 2)

        if sucesso_api and response:
            areas_medicas = ['Não Classificado']
            if response.parts:
                cleaned_text = response.text.strip()
                if cleaned_text.startswith("```python"):
                    cleaned_text = cleaned_text[len("```python"):].strip()
                elif cleaned_text.startswith("```"):
                     cleaned_text = cleaned_text[len("```"):].strip()
                if cleaned_text.endswith("```"):
                    cleaned_text = cleaned_text[:-len("```")].strip()

                try:
                    parsed_list = ast.literal_eval(cleaned_text)
                    if isinstance(parsed_list, list) and all(isinstance(item, str) for item in parsed_list):
                        areas_medicas = [area.strip() for area in parsed_list if area.strip()]
                        if not areas_medicas:
                             areas_medicas = ['Não Classificado - Lista Vazia']
                             erros_parse += 1
                    else:
                         print(f" [AVISO: Formato de resposta inesperado após limpeza: {cleaned_text}. Tentando parse alternativo.]", end="")
                         areas_medicas = [area.strip().strip("'\"[] ") for area in cleaned_text.split(',') if area.strip()]
                         if not areas_medicas:
                             areas_medicas = ['Não Classificado - Erro Parse Alternativo']
                             erros_parse += 1

                except (ValueError, SyntaxError, TypeError) as parse_error:
                    print(f" [ERRO ao parsear resposta da IA após limpeza: {parse_error}. Resposta original: '{response.text.strip()}', Limpa: '{cleaned_text}']", end="")
                    areas_medicas = ['Não Classificado - Erro Parse']
                    erros_parse += 1
                except Exception as e:
                     print(f" [ERRO inesperado no parse: {e}. Resposta original: '{response.text.strip()}', Limpa: '{cleaned_text}']", end="")
                     areas_medicas = ['Não Classificado - Erro Parse Inesperado']
                     erros_parse += 1

            else:
                 block_reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Razão desconhecida"
                 print(f" [AVISO: Resposta vazia ou bloqueada pela IA. Razão: {block_reason}]", end="")
                 areas_medicas = ['Não Classificado - Bloqueado']
                 erros_api += 1

            questao['areas_medicas'] = areas_medicas
            print(f" -> {areas_medicas}")
            area_counter.update(areas_medicas)
            questoes_processadas += 1

        time.sleep(PAUSA_ENTRE_REQUISICOES)

    # --- PASSO 5: Salvar o Novo JSON (APENAS os dados filtrados e processados) ---
    print(f"\nSalvando os resultados classificados (apenas ano {target_year}) em: {output_file_path}")
    try:
        # --- MODIFICAÇÃO: Salvar data_to_process ---
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(data_to_process, f, ensure_ascii=False, indent=4)
        print("Arquivo salvo com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar o arquivo JSON de saída: {e}")

    # --- PASSO 6: Gerar Resumo ---
    print("\n--- Resumo da Classificação por Área (Apenas Ano 2024) ---")
    print(f"Total de questões de 2024 encontradas: {total_questoes_filtradas}")
    print(f"Questões processadas com sucesso pela API: {questoes_processadas}")
    print(f"Questões com erro na chamada da API (Quota/Conexão/Outros): {erros_api}")
    print(f"Questões com erro no parse da resposta da IA: {erros_parse}")
    print(f"Questões com enunciado vazio: {len([q for q in data_to_process if q.get('areas_medicas') == ['Não Classificado - Enunciado Vazio']])}")
    print("-" * 20)
    if area_counter:
        for area, contagem in area_counter.most_common():
            print(f"{area}: {contagem} questões")
    else:
        print("Nenhuma classificação foi realizada com sucesso.")

elif not all_data:
     print("\nNenhum dado foi carregado do arquivo JSON. Classificação e salvamento cancelados.")
# Se all_data foi carregado mas data_to_process ficou vazio
elif not data_to_process:
     print(f"\nNenhuma questão encontrada para o ano {target_year}. Nenhum processamento ou salvamento realizado.")

API Key carregada com sucesso.

Tentando ler o arquivo de entrada: /content/questoes-Revalida+Fuvest.json
Arquivo '/content/questoes-Revalida+Fuvest.json' lido com sucesso. 1301 questões encontradas no total.

Filtrado para o ano 2024. 312 questões encontradas para processar.

Iniciando classificação das questões de 2024...
Classificando questão 1/312 (Prova 2024 - Nº 1)... -> ['Clínica Médica', 'Urologia', 'Ginecologia']
Classificando questão 2/312 (Prova 2024 - Nº 2)... -> ['Urgência/Emergência', 'Cirurgia Geral', 'Urologia']
Classificando questão 3/312 (Prova 2024 - Nº 3)... -> ['Pediatria', 'Neurologia', 'Oftalmologia']
Classificando questão 4/312 (Prova 2024 - Nº 4)... -> ['Obstetrícia', 'Ginecologia', 'Urgência/Emergência']
Classificando questão 5/312 (Prova 2024 - Nº 5)... -> ['Preventiva/Saúde Coletiva', 'Medicina de Família e Comunidade', 'Ética Médica']
Classificando questão 6/312 (Prova 2024 - Nº 6)... -> ['Clínica Médica Geral', 'Preventiva/Saúde Coletiva', 'Endocrinologia'